In [1]:
from rdkit import Chem

In [2]:
import pickle as pkl
from glob import glob

In [5]:
def load_pkl_pred(input_glob):
    FF = glob(input_glob)

    merged_data = []

    for fn in FF:
        with open(fn, 'rb') as f:
            data = pkl.load(f)
            merged_data.append([rec for rec in data if 'This is a padding sentence.' not in rec[0]])
        print(f'- {fn}: {len(data)}')

    print(len(merged_data))
    return merged_data

# moe_pred = load_pkl_pred('/nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/*.pkl')
# moe_pred = load_pkl_pred('/nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/uspto50k/*.pkl')
moe_pred = load_pkl_pred('/nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/*.pkl')


- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part0.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part1.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part10.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part11.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part12.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part13.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part14.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part15.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part2.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part3.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part4.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part5.pkl: 31

In [7]:
moe_pred[0][0]

('What reactants could possibly lead to the formation of this product? <product>S(c1c(OCC#C)cc(OC)cc1OC)(=O)(=O)OCC(C([C@@H](NC([C@@H](NC([C@H](CC(C)C)NC(=O)[C@H](CCc1ccccc1)NC(=O)CN1CCOCC1)=O)Cc1ccccc1)=O)CC(C)C)=O)(C)O</product>\t<reactants>S(c1c(OCC#C)cc(OC)cc1OC)(=O)(=O)Cl.OC[C@@](C([C@@H](NC([C@@H](NC([C@H](CC(C)C)NC(=O)[C@H](CCc1ccccc1)NC(=O)CN1CCOCC1)=O)Cc1ccccc1)=O)CC(C)C)=O)(C)O</reactants>',
 ['<reactants> <m>S <m>( <m>c <m>1 <m>c <m>( <m>O <m>C <m>C <m># <m>C <m>) <m>c <m>c <m>( <m>O <m>C <m>) <m>c <m>c <m>1 <m>O <m>C <m>) <m>( <m>= <m>O <m>) <m>( <m>= <m>O <m>) <m>Cl <m>. <m>O <m>C <m>C <m>( <m>C <m>( <m>[C@@H] <m>( <m>N <m>C <m>( <m>[C@@H] <m>( <m>N <m>C <m>( <m>[C@H] <m>( <m>C <m>C <m>( <m>C <m>) <m>C <m>) <m>N <m>C <m>( <m>= <m>O <m>) <m>[C@H] <m>( <m>C <m>C <m>c <m>1 <m>c <m>c <m>c <m>c <m>c <m>1 <m>) <m>N <m>C <m>( <m>= <m>O <m>) <m>C <m>N <m>1 <m>C <m>C <m>O <m>C <m>C <m>1 <m>) <m>= <m>O <m>) <m>C <m>c',
  '<reactants> <m>S <m>( <m>c <m>1 <m>c <m>( <m>O <m>C <m>C <m>#

In [8]:
def flaten_pkl_pred(data):
    ret = []
    folds = len(data)
    max_fold_size = max([len(f) for f in data])
    
    for i in range(max_fold_size):
        for j in range(folds):
            if i < len(data[j]):
                rec = data[j][i]
                for sent in rec[1]:
                    sent = sent.replace(' <m>', '').replace(' ', '').strip()
                    ret.append(sent)

    print('total:', len(ret))
    return ret

flaten_pkl_pred(moe_pred)[0]

total: 20028


'<reactants>S(c1c(OCC#C)cc(OC)cc1OC)(=O)(=O)Cl.OCC(C([C@@H](NC([C@@H](NC([C@H](CC(C)C)NC(=O)[C@H](CCc1ccccc1)NC(=O)CN1CCOCC1)=O)Cc'

In [10]:
def canno(m):
    for tag in ['<product>', '</product>', '<reactants>', '</reactants>', '<reagents>', '</reagents>']:
        m = m.replace(tag, '')
    m = m.strip()
    try:
        return Chem.MolToSmiles(Chem.MolFromSmiles(m))
    except:
        return ''

def compute_top_k_acc(labels, preds, k=4):
    print('labels:', labels[:10])
    print('preds:', preds[:10])
    
    
    correct_at_k = [0]*k
    
    for i in range(len(labels)):
        label = canno(labels[i])
        correct = False
        for j in range(k):
            pred =  canno(preds[i*k+j])
            if label == pred:
                correct = True
            if correct:
                correct_at_k[j] += 1

    return [x/len(labels) for x in correct_at_k]

def compute_top_k_acc_from_file(label_file, pred_file, k=4):
    labels = []
    preds = []
    with open(label_file, 'r') as f:
        for line in f:
            labels.append(line.strip().split('\t')[1])
    
    with open(pred_file, 'r') as f:
        for line in f:
            preds.append(line.strip())

    return compute_top_k_acc(labels, preds, k)

def compute_top_k_acc_from_pickle(label_file, pred_file, k=4):
    labels = []
    preds = []
    with open(label_file, 'r') as f:
        for line in f:
            labels.append(line.strip().split('\t')[1])
    
    preds_raw = load_pkl_pred(pred_file)
    preds = flaten_pkl_pred(preds_raw)

    return compute_top_k_acc(labels, preds, k)

# compute_top_k_acc_from_file(
#     '/blob/yinxia/wu2/shared/SFM/SFM.overall.data/SFMMolInstruct.20240617.test/test.uspto50k.reaction.osmi.tsv',
#     '/home/shufxi/workspace/SFM_NLM_resources/all_infer_results/8x7b_global_step33216_infer_results/uspto50k.reaction.osmi'
# )

compute_top_k_acc_from_pickle(
    '/blob/yinxia/wu2/shared/SFM/SFM.overall.data/SFMMolInstruct.20240617.test/test.uspto50k.retro.osmi.tsv',
    # '/nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/uspto50k/*.pkl'
    '/nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/*.pkl'
)

- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part0.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part1.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part10.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part11.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part12.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part13.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part14.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part15.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part2.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part3.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part4.pkl: 313
- /nlm/shufxi/output/inference/retro/mixtral_8x7b_mp/20240705132320/part5.pkl: 31

[09:50:40] SMILES Parse Error: extra open parentheses for input: 'S(c1c(OCC#C)cc(OC)cc1OC)(=O)(=O)Cl.OCC(C([C@@H](NC([C@@H](NC([C@H](CC(C)C)NC(=O)[C@H](CCc1ccccc1)NC(=O)CN1CCOCC1)=O)Cc'
[09:50:40] SMILES Parse Error: extra open parentheses for input: 'S(c1c(OCC#C)cc(OC)cc1OC)(=O)(=O)Cl.OCC(C(C(NC([C@@H](NC([C@H](CC(C)C)NC(=O)[C@H](CCc1ccccc1)NC(=O)CN1CCOCC1)=O)Cc'
[09:50:40] SMILES Parse Error: extra open parentheses for input: 'S(c1c(OCC#C)cc(OC)cc1OC)(=O)(=O)Cl.OCC(C(C(NC(C(NC([C@H](CC(C)C)NC(=O)[C@H](CCc1ccccc1)NC(=O)CN1CCOCC1)=O)Cc'
[09:50:40] SMILES Parse Error: extra open parentheses for input: 'S(c1c(OCC#C)cc(OC)cc1OC)(=O)(=O)Cl.OCC(C([C@H](NC([C@@H](NC([C@H](CC(C)C)NC(=O)[C@H](CCc1ccccc1)NC(=O)CN1CCOCC1)=O)Cc'
[09:50:40] SMILES Parse Error: syntax error while parsing: S(c1ncn(C(N(C)C)=O)n1)(=O)(=O)Cl.C1CCOC1.CCN(CC)CC</mol>
[09:50:40] SMILES Parse Error: Failed parsing SMILES 'S(c1ncn(C(N(C)C)=O)n1)(=O)(=O)Cl.C1CCOC1.CCN(CC)CC</mol>' for input: 'S(c1ncn(C(N(C)C)=O)n1)(=O)(=O)Cl

[0.00019972039145196724,
 0.00019972039145196724,
 0.00019972039145196724,
 0.00019972039145196724]

In [20]:
! head '/blob/yinxia/wu2/shared/SFM/SFM.overall.data/SFMMolInstruct.20240617.test/test.uspto50k.retro.osmi.tsv'

Provided the product below, propose some possible reactants that could have been used in the reaction. <product>CC(C)(C)OC(=O)n1c2ccc(C(C)=O)cc2cc1</product>	<reactants>CC(C)(C)OC(=O)OC(=O)OC(C)(C)C.[nH]1c2ccc(C(C)=O)cc2cc1</reactants>
Which reactants could have been used to generate the given product? <product>CC(C)(C)OC(=O)N1C[C@@H](OS(c2ccc(C)cc2)(=O)=O)[C@@H]2[C@H]1[C@@H](O)CO2</product>	<reactants>CC(C)(C)OC(=O)OC(=O)OC(C)(C)C.N1C[C@@H](OS(c2ccc(C)cc2)(=O)=O)[C@@H]2[C@H]1[C@@H](O)CO2</reactants>
Please suggest possible reactants for the given product. <product>BrCc1c(C(OCC)=O)nn(-c2ccc(Cl)cc2Cl)c1-c1ccc(OC)cc1</product>	<reactants>BrN1C(=O)CCC1=O.Cc1c(C(OCC)=O)nn(-c2ccc(Cl)cc2Cl)c1-c1ccc(OC)cc1</reactants>
Can you propose some plausible reactants that could have resulted in this product? <product>CC(C)(C)OC(=O)Nc1nc2c(-c3cccc([N+](=O)[O-])c3)cc(C#N)cc2s1</product>	<reactants>CC(C)(C)OC(=O)OC(=O)OC(C)(C)C.Nc1nc2c(-c3cccc([N+](=O)[O-])c3)cc(C#N)cc2s1</reactants>
Predict possible rea

In [21]:
moe_pred[0]

('Provided the product below, propose some possible reactants that could have been used in the reaction. <product>CC(C)(C)OC(=O)n1c2ccc(C(C)=O)cc2cc1</product>\t<reactants>CC(C)(C)OC(=O)OC(=O)OC(C)(C)C.[nH]1c2ccc(C(C)=O)cc2cc1</reactants>',
 ['<reactants> <m>C <m>C <m>( <m>C <m>) <m>( <m>C <m>) <m>O <m>C <m>( <m>= <m>O <m>) <m>n <m>1 <m>c <m>2 <m>c <m>c <m>c <m>c <m>c <m>2 <m>c <m>c <m>1 <m>. <m>C <m>( <m>C <m>) <m>( <m>= <m>O <m>) <m>Cl </reactants>',
  '<reactants> <m>C <m>C <m>( <m>C <m>) <m>( <m>C <m>) <m>O <m>C <m>( <m>= <m>O <m>) <m>O <m>C <m>( <m>= <m>O <m>) <m>O <m>C <m>( <m>C <m>) <m>( <m>C <m>) <m>C <m>. <m>[nH] <m>1 <m>c <m>2 <m>c <m>c <m>c <m>( <m>C <m>( <m>C <m>) <m>= <m>O <m>) <m>c <m>c <m>2 <m>c <m>c <m>1 </reactants>',
  '<reactants> <m>C <m>C <m>( <m>C <m>) <m>( <m>C <m>) <m>O <m>C <m>( <m>= <m>O <m>) <m>O <m>C <m>( <m>O <m>C <m>( <m>C <m>) <m>( <m>C <m>) <m>C <m>) <m>= <m>O <m>. <m>[nH] <m>1 <m>c <m>2 <m>c <m>c <m>c <m>( <m>C <m>( <m>C <m>) <m>= <m>O <m>) <m>c <m>c <m

In [25]:
len(flaten_pkl_pred(moe_pred)[0])

total: 20028


61